In [31]:
from EMS import EMS
from Organization import Organization
from Energy import Energy
from Hierarchy import Hierarchy
from Dataflow import DataFlow
import pandas as pd
import numpy as np
import pandas as pd

from EMS import EMS
from Organization import Organization
from Energy import Energy
from Hierarchy import Hierarchy
from Dataflow import DataFlow

import xml.etree.ElementTree as ET
from common import readOption, logger
from IPython.display import display
# Set the maximum number of columns to be displayed to None
pd.set_option('display.max_columns', None)

In [32]:
code = readOption("code")
components_binding = readOption("components_binding")        
host=readOption("databases.organization.host")
port=readOption("databases.organization.port")
database=readOption("databases.organization.database")
user=readOption("databases.organization.username")
password=readOption("databases.organization.password")
org = Organization(host=host, port=port, user=user, password=password, database=database)

tenant = org.Tenant(code)

print(f"{tenant.name.values[0]} | {tenant.company_code.values[0]} | {tenant.id.values[0]}")

Canadian International School | cdnis | 9012847f-290f-5e49-a1c7-d9fa78f9eb3e


In [33]:
# generate tenant tree in dataflow
dataFlow = DataFlow(code, components_binding, simulation=True)
df = dataFlow.PreparingData()
# df = dataFlow.LoadData()
logger.info("====================== create_nodes_and_datapoints ==========================")
dataFlow.create_nodes_and_datapoints(df, tenant.id.values[0])

/home/zhixiang/EnTrak/Project/ems-hierarchy-energy/Hierarchy.py:466: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.simulation_relations_df = self.simulation_relations_df.append(new_row, ignore_index=True)
/home/zhixiang/EnTrak/Project/ems-hierarchy-energy/Hierarchy.py:466: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.simulation_relations_df = self.simulation_relations_df.append(new_row, ignore_index=True)
/home/zhixiang/EnTrak/Project/ems-hierarchy-energy/Hierarchy.py:466: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.simulation_relations_df = self.simulation_relations_df.append(new_row, ignore_index=True)
/home/zhixiang/EnTrak/Project/ems-hierarchy-energy/Hierarchy.py:466: FutureWarning: The frame.append me

In [34]:
ssd = dataFlow.hr.simulation_sys_df
print(ssd.shape)
with pd.option_context('display.max_columns', None):
    display(ssd[:2])

(33, 25)


,id_x,parent_system_id,name_x,source_key,meter_id,composition_expression,component_of_id,company_id,city_id,city_name,id_y,ref_id,name_y,tenant_id,company,company_code,index,use_datapoint_id,use_system_id,node_id,node_type,data_type,node_ref_id,component,expression_replaced
0,363,NaN,Canadian International School,None,NaN,None,NaN,142,2,Hong Kong,NaN,NaN,NaN,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis,0,NaN,NaN,fbfbb77a-e00c-46b8-8de6-c0acf5253964,SITE,unknown,unknown,0,0
1,379,367.0,Classrooms,,NaN,None,NaN,142,2,Hong Kong,NaN,NaN,NaN,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis,1,NaN,NaN,45a95c22-d9be-4aa5-8ef6-6601738c4d25,POV,unknown,unknown,0,0


In [35]:
srd = dataFlow.hr.simulation_relations_df
print(srd.shape)
with pd.option_context('display.max_columns', None):
    display(srd[:10])

(14, 4)


,parent_id,parent_type,child_id,child_type
0,8a2dbd62-e41e-418c-a02a-55f118e726ed,POV,45a95c22-d9be-4aa5-8ef6-6601738c4d25,POV
1,fbfbb77a-e00c-46b8-8de6-c0acf5253964,SITE,a5871f05-6e87-4bb5-a475-108b4bab3b7d,POV
2,45a95c22-d9be-4aa5-8ef6-6601738c4d25,POV,d1a54e8f-b14f-40cb-be5e-e63a231114e0,DATAPOINT
3,fbfbb77a-e00c-46b8-8de6-c0acf5253964,SITE,8a2dbd62-e41e-418c-a02a-55f118e726ed,POV
4,8a2dbd62-e41e-418c-a02a-55f118e726ed,POV,dcc4b43f-19c7-499f-a062-ca04d6740087,DATAPOINT
5,a5871f05-6e87-4bb5-a475-108b4bab3b7d,POV,c1218502-3c52-4ec2-86fa-a716d4f33fa4,DATAPOINT
6,45a95c22-d9be-4aa5-8ef6-6601738c4d25,POV,305e62e9-3252-488f-abbf-a4ab0f4613ba,DATAPOINT
7,cbefd859-0bd5-49d0-a2af-1d76c2004b49,POV,44cab3ed-637c-4e00-ace2-813d18d3c413,DATAPOINT
8,8a2dbd62-e41e-418c-a02a-55f118e726ed,POV,c4330ec6-a816-45d2-8540-c5fde82d48c3,DATAPOINT
9,6deaa85b-d33d-409e-9d70-d7e10e8880d4,POV,85630218-1698-4d74-98f6-3a52547183e0,DATAPOINT


In [36]:
site_node_id = srd[srd['parent_type']=='SITE']['parent_id'].iloc[0]
print(site_node_id)

fbfbb77a-e00c-46b8-8de6-c0acf5253964


In [37]:
hr = dataFlow.hr
logger.debug("====================== Simulate TenantTree XML ==========================")    
tenantTree = dataFlow.hr.TenantSimulateTree(
                        # tenant.id.values[0],
                        site_node_id,
                        tenant.name.values[0],
                        tenant.company_code.values[0])

site_path = f"./output/{code}"
hr.SaveToXml(tenantTree, f"{site_path}/sim_new_{code}.xml") 

/home/zhixiang/EnTrak/Project/ems-hierarchy-energy/Hierarchy.py:256: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relations_df['parent_id'] = relations_df['parent_id'].astype(str)
/home/zhixiang/EnTrak/Project/ems-hierarchy-energy/Hierarchy.py:257: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relations_df['child_id'] = relations_df['child_id'].astype(str)
/home/zhixiang/EnTrak/Project/ems-hierarchy-energy/Hierarchy.py:277: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [38]:
host=readOption("databases.ems.host")
port=readOption("databases.ems.port")
database=readOption("databases.ems.database")
user=readOption("databases.ems.username")
password=readOption("databases.ems.password")    

logger.debug("======================= EMS system tree XML =========================")
ems = EMS(host=host, port=port, user=user, password=password, database=database)
company = ems.company(code)

# logger.debug(f"{company.name.values[0]} | {company.code.values[0]} | {company.id.values[0]}")
companyTree = ems.SystemTree(company.id.values[0],
                                company.name.values[0],
                                company.code.values[0])

ems.SaveToXml(companyTree, f"{site_path}/sim_{code}_system.xml")

In [39]:
ems = EMS("localhost", "5432", "mangopie", "mangopie", "mangopie")

In [40]:
company_df = ems.company(code="cdnis")
print(company_df)

    id                           name   code
0  142  Canadian International School  cdnis


In [41]:
org = Organization()
tenant_df = org.Tenant(code="cdnis")

print(tenant_df)

                                     id                           name  \
0  9012847f-290f-5e49-a1c7-d9fa78f9eb3e  Canadian International School   

  company_code  
0        cdnis  


In [42]:
joined_tenant = pd.merge(company_df, tenant_df, how="left", left_on="code", right_on="company_code")
tenant_df = joined_tenant[['id_x', 'id_y', 'name_x', 'code']]

new_names = {'id_x': 'company_id', 'id_y': 'tenant_id', 'name_x': 'company', "code": 'company_code'}
tenant_df = tenant_df.rename(columns=new_names)

with pd.option_context('display.max_columns', None):
    display(tenant_df[:3])   

,company_id,tenant_id,company,company_code
0,142,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis


In [43]:
sys_df = ems.systems(code="cdnis")
# Set options temporarily within the context of `display()`
with pd.option_context('display.max_columns', None):
    display(sys_df[:8])

,id,parent_system_id,name,source_key,meter_id,composition_expression,component_of_id,company_id,city_id,city_name
0,363,NaN,Canadian International School,None,NaN,None,NaN,142,2,Hong Kong
1,379,367.0,Classrooms,,NaN,None,NaN,142,2,Hong Kong
2,364,363.0,Leo Lee Arts Centre,,NaN,None,NaN,142,2,Hong Kong
3,384,NaN,MR,MR,49.0,None,381.0,142,2,Hong Kong
4,385,379.0,Upper Sch CR & Hallways (5-8F),,NaN,1*{id_389} + 1*{id_387} + -1*{id_388} + 1*{id_...,NaN,142,2,Hong Kong
5,375,NaN,1000a,1000a,50.0,None,368.0,142,2,Hong Kong
6,376,NaN,1250a,1250a,50.0,None,368.0,142,2,Hong Kong
7,374,NaN,630a,630a,50.0,None,368.0,142,2,Hong Kong


In [44]:
# sys_df = ems.virtual_systems(code="cdnis")
# # Set options temporarily within the context of `display()`
# with pd.option_context('display.max_columns', None):
#     display(sys_df[:8])

In [45]:
energy = Energy("localhost", "5432", "energy", "energy", "energy")

In [46]:
dp = energy.dataPoint(columns = ["id", "ref_id", "name"])
dp[:3]

,id,ref_id,name
0,e7b7c78b-4ec2-4d75-b7c6-5c25e6ef61c5,520,Special Block AC
1,9a6c367b-5574-47be-bdb1-0ee6838ac61c,16053,socket and other power 23F
2,35a562bf-0e4e-47aa-95cd-76f6cafc0aaf,14463,isf_4NRLP8_4100


In [47]:
joined_df = pd.merge(sys_df, dp, how="left", left_on="id", right_on="ref_id")


# 查看连接后的DataFrame
with pd.option_context('display.max_columns', None):
    display(joined_df[:8])


,id_x,parent_system_id,name_x,source_key,meter_id,composition_expression,component_of_id,company_id,city_id,city_name,id_y,ref_id,name_y
0,363,NaN,Canadian International School,None,NaN,None,NaN,142,2,Hong Kong,NaN,NaN,NaN
1,379,367.0,Classrooms,,NaN,None,NaN,142,2,Hong Kong,NaN,NaN,NaN
2,364,363.0,Leo Lee Arts Centre,,NaN,None,NaN,142,2,Hong Kong,NaN,NaN,NaN
3,384,NaN,MR,MR,49.0,None,381.0,142,2,Hong Kong,NaN,NaN,NaN
4,385,379.0,Upper Sch CR & Hallways (5-8F),,NaN,1*{id_389} + 1*{id_387} + -1*{id_388} + 1*{id_...,NaN,142,2,Hong Kong,7436449a-c21b-4a79-8d0d-f1934afe734d,385.0,
5,375,NaN,1000a,1000a,50.0,None,368.0,142,2,Hong Kong,cd445559-331a-4730-8d6b-de1558e90f4c,375.0,1000a
6,376,NaN,1250a,1250a,50.0,None,368.0,142,2,Hong Kong,5ed29e24-b5d2-4b53-b486-ec5a707c03f0,376.0,1250a
7,374,NaN,630a,630a,50.0,None,368.0,142,2,Hong Kong,3ccd5b3c-e3db-49e8-ad08-51b9be9e673a,374.0,630a


In [48]:
joined_df.loc[joined_df['id_x'] == int(386)]#[['id_y', 'ref_id']]

,id_x,parent_system_id,name_x,source_key,meter_id,composition_expression,component_of_id,company_id,city_id,city_name,id_y,ref_id,name_y
19,386,NaN,Upper Sch CR & Hallways (5-8F),ML,49.0,None,385.0,142,2,Hong Kong,NaN,NaN,NaN


In [49]:
joined_df.loc[joined_df['source_key'] == 'ML']

,id_x,parent_system_id,name_x,source_key,meter_id,composition_expression,component_of_id,company_id,city_id,city_name,id_y,ref_id,name_y
11,371,NaN,ML,ML,49.0,None,368.0,142,2,Hong Kong,c3387410-6c7a-471d-a9fa-45afe64d468e,371.0,ML
19,386,NaN,Upper Sch CR & Hallways (5-8F),ML,49.0,None,385.0,142,2,Hong Kong,NaN,NaN,NaN


In [50]:
with pd.option_context('display.max_columns', None):
    display(tenant_df[:3])   

,company_id,tenant_id,company,company_code
0,142,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis


In [51]:
joined_df = pd.merge(joined_df, tenant_df, how="left", left_on="company_id", right_on="company_id")
# 查看连接后的DataFrame
with pd.option_context('display.max_columns', None):
    display(joined_df[:8])

,id_x,parent_system_id,name_x,source_key,meter_id,composition_expression,component_of_id,company_id,city_id,city_name,id_y,ref_id,name_y,tenant_id,company,company_code
0,363,NaN,Canadian International School,None,NaN,None,NaN,142,2,Hong Kong,NaN,NaN,NaN,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis
1,379,367.0,Classrooms,,NaN,None,NaN,142,2,Hong Kong,NaN,NaN,NaN,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis
2,364,363.0,Leo Lee Arts Centre,,NaN,None,NaN,142,2,Hong Kong,NaN,NaN,NaN,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis
3,384,NaN,MR,MR,49.0,None,381.0,142,2,Hong Kong,NaN,NaN,NaN,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis
4,385,379.0,Upper Sch CR & Hallways (5-8F),,NaN,1*{id_389} + 1*{id_387} + -1*{id_388} + 1*{id_...,NaN,142,2,Hong Kong,7436449a-c21b-4a79-8d0d-f1934afe734d,385.0,,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis
5,375,NaN,1000a,1000a,50.0,None,368.0,142,2,Hong Kong,cd445559-331a-4730-8d6b-de1558e90f4c,375.0,1000a,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis
6,376,NaN,1250a,1250a,50.0,None,368.0,142,2,Hong Kong,5ed29e24-b5d2-4b53-b486-ec5a707c03f0,376.0,1250a,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis
7,374,NaN,630a,630a,50.0,None,368.0,142,2,Hong Kong,3ccd5b3c-e3db-49e8-ad08-51b9be9e673a,374.0,630a,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis


In [52]:
def replace_id(df):
  """
  循环替换 DataFrame 中的 {id_xxx}

  Args:
    df: 输入 DataFrame

  Returns:
    替换后的 DataFrame
  """
  import re
  for i in range(df.shape[0]):
    row = df.iloc[i]
    composition_expression = row['composition_expression']
    for match in re.finditer(r'{id_(\d+)}', composition_expression):
      id_xxx = match.group(1)
      use_system_id = df.loc[df['id'] == int(id_xxx), 'use_system_id'].values[0]
      composition_expression = composition_expression.replace(match.group(0), "{id_"+str(use_system_id)+"}")
    df.loc[i, 'composition_expression'] = composition_expression

  return df

# 使用示例
df = pd.DataFrame({
  'id': [13, 63, 8],
  'composition_expression': [
    '1 if  {id_13}  >= 3 else 1 if  {id_63}  >= 3 else {id_8}',
    '1 if ({id_63}*2)+({id_63}*{id_8})  >= 3 else {id_8}',
    '{id_8}'
  ],
  'use_system_id': [100, 200, 300]
})

print(df)
print("----------------------------------------------------------------")
df = replace_id(df)

print(df)


   id                             composition_expression  use_system_id
0  13  1 if  {id_13}  >= 3 else 1 if  {id_63}  >= 3 e...            100
1  63  1 if ({id_63}*2)+({id_63}*{id_8})  >= 3 else {...            200
2   8                                             {id_8}            300
----------------------------------------------------------------
   id                             composition_expression  use_system_id
0  13  1 if  {id_100}  >= 3 else 1 if  {id_200}  >= 3...            100
1  63  1 if ({id_200}*2)+({id_200}*{id_300})  >= 3 el...            200
2   8                                           {id_300}            300


In [53]:
vdp = energy.virtualDataPoint()
vdp[:3]

,datapoint_id,expression,name
0,4b4a78d4-7bce-415f-b7f3-87faf3248b8d,{id_6002}-{id_5993}-{id_5994}-{id_5996}-{id_59...,HP-3
1,a271606e-e62a-4d42-9fdf-bf4ac890cec4,{id_514}+{id_515},3/F Lights & Sockets
2,4c5d9f40-b2d4-44af-a896-fcb0bf9409b3,na,test


In [54]:
sys_df

,id,parent_system_id,name,source_key,meter_id,composition_expression,component_of_id,company_id,city_id,city_name
0,363,NaN,Canadian International School,None,NaN,None,NaN,142,2,Hong Kong
1,379,367.0,Classrooms,,NaN,None,NaN,142,2,Hong Kong
2,364,363.0,Leo Lee Arts Centre,,NaN,None,NaN,142,2,Hong Kong
3,384,NaN,MR,MR,49.0,None,381.0,142,2,Hong Kong
4,385,379.0,Upper Sch CR & Hallways (5-8F),,NaN,1*{id_389} + 1*{id_387} + -1*{id_388} + 1*{id_...,NaN,142,2,Hong Kong
5,375,NaN,1000a,1000a,50.0,None,368.0,142,2,Hong Kong
6,376,NaN,1250a,1250a,50.0,None,368.0,142,2,Hong Kong
7,374,NaN,630a,630a,50.0,None,368.0,142,2,Hong Kong
8,370,NaN,B,B,49.0,None,368.0,142,2,Hong Kong
9,377,NaN,Canteen3,Canteen3,76.0,None,368.0,142,2,Hong Kong


In [55]:
hr = Hierarchy()

In [56]:
n_dp = hr.nodeDataPoint()

with pd.option_context('display.max_columns', None):
    display(n_dp[:2])

,id,name,data_type,data_id
0,7c2f269a-b04b-43e7-b651-094721829fbe,MR,ENERGY,435fc64c-252c-4655-a3e8-4468a655a0d2
1,3f46e815-54ec-440b-8766-da42b1850842,Charging station,ENERGY,98298dd1-9fb7-41a7-83d9-89580aefe661


In [57]:
n_pov = hr.nodePov()
with pd.option_context('display.max_columns', None):
    display(n_pov[:2])

,id,name
0,f8eac8a7-7985-4d29-8e45-157bd85edf33,IAQ Sensor
1,a99073ad-6a2f-49f8-be5d-3912387662aa,Classrooms


In [58]:
n_site = hr.nodeSite()
with pd.option_context('display.max_columns', None):
    display(n_site[:2])

,id,city_id,name
0,921aecbf-d810-4c31-ba72-fa5d90974927,c56d9525-5c54-4486-af7d-59bb1189fc45,Hong Kong Smart School Demo
1,ef2973c8-c885-4501-9b5b-1a090dd4b8d4,c56d9525-5c54-4486-af7d-59bb1189fc45,Canadian International School


In [59]:
n_relations = hr.nodeRelations()
with pd.option_context('display.max_columns', None):
    display(n_relations[:2])

,id,parent_id,parent_type,child_id,child_type
0,1330,a41c3508-9da8-4a04-8d11-66c0c39d249d,POV,97b03982-1a0f-494d-a3a2-7aee5c72b52a,DATAPOINT
1,1331,4465377a-f87a-570e-98d7-084930e4e21d,TENANT,921aecbf-d810-4c31-ba72-fa5d90974927,SITE


In [60]:
# tenantTree = hr.TenantTree("c93789d6-d78e-5382-8817-c3917ca879be")
# tenantTree